2. Простой пример на Python
Транспортная компания отправляет грузы разными способами.

In [1]:
from abc import ABC, abstractmethod

# ---------- Product ----------
class Transport(ABC):
    @abstractmethod
    def deliver(self, package_id: str) -> None:
        """Отправить груз."""
        ...

class Truck(Transport):
    def deliver(self, package_id: str) -> None:
        print(f"📦 Груз {package_id} везём грузовиком по дороге")

class Ship(Transport):
    def deliver(self, package_id: str) -> None:
        print(f"🚢 Груз {package_id} отправляем кораблём по морю")

# ---------- Creator ----------
class Logistics(ABC):
    """В этой базовой логике мы не знаем КАК доставлять — только ЧТО делать."""
    
    def plan_delivery(self, package_id: str) -> None:
        transport = self.create_transport()      # Фабричный метод
        transport.deliver(package_id)

    @abstractmethod
    def create_transport(self) -> Transport:
        ...

class RoadLogistics(Logistics):
    """Конкретная фабрика: выбирает Truck."""
    def create_transport(self) -> Transport:
        return Truck()

class SeaLogistics(Logistics):
    """Конкретная фабрика: выбирает Ship."""
    def create_transport(self) -> Transport:
        return Ship()

# ---------- Клиентский код ----------
if __name__ == "__main__":
    for logistics in (RoadLogistics(), SeaLogistics()):
        logistics.plan_delivery("ORDER‑42")


📦 Груз ORDER‑42 везём грузовиком по дороге
🚢 Груз ORDER‑42 отправляем кораблём по морю


Клиент (plan_delivery) не знает, Truck это или Ship. Он просто вызывает deliver().

3. Сложный пример: платёжная система с расширяемыми провайдерами
Задача: интернет‑магазин хочет принимать PayPal, Stripe, Crypto … и в будущем легко добавлять новые способы.

Диаграмма

classDiagram
    class PaymentProcessor{
        +process(amount)
    }
    class PayPalProcessor
    class StripeProcessor
    class CryptoProcessor

    class CheckoutService{
        +pay(amount)
        <<Creator>>
        +create_processor()
    }
    class USCheckoutService
    class EUEasyCheckoutService

    PaymentProcessor <|-- PayPalProcessor
    PaymentProcessor <|-- StripeProcessor
    PaymentProcessor <|-- CryptoProcessor
    CheckoutService <|-- USCheckoutService
    CheckoutService <|-- EUEasyCheckoutService
    CheckoutService --> PaymentProcessor : uses


In [2]:
"""
Расширяемая платёжная система. 
Добавление нового провайдера = новый класс + регистрация, без правки остального кода.
"""
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Type, Dict

# ---------- Product ----------
class PaymentProcessor(ABC):
    @abstractmethod
    def process(self, amount: float) -> None:
        ...

# ---------- Concrete Products ----------
class PayPalProcessor(PaymentProcessor):
    def process(self, amount: float) -> None:
        print(f"💰 PayPal списал ${amount:.2f}")

class StripeProcessor(PaymentProcessor):
    def process(self, amount: float) -> None:
        print(f"💳 Stripe обработал ${amount:.2f}")

class CryptoProcessor(PaymentProcessor):
    def process(self, amount: float) -> None:
        print(f"₿ Криптокошелёк получил эквивалент ${amount:.2f}")

# ---------- Реестр провайдеров (необязательный, но удобный «трюк») ----------
_PROCESSOR_REGISTRY: Dict[str, Type[PaymentProcessor]] = {}

def register_processor(name: str):
    """Декоратор для автоматической регистрации нового процессора."""
    def decorator(cls: Type[PaymentProcessor]):
        _PROCESSOR_REGISTRY[name] = cls
        return cls
    return decorator

# Применим декоратор
register_processor("paypal")(PayPalProcessor)
register_processor("stripe")(StripeProcessor)
register_processor("crypto")(CryptoProcessor)

# ---------- Creator ----------
class CheckoutService(ABC):
    def __init__(self, provider_name: str):
        self._provider_name = provider_name.lower()

    def pay(self, amount: float) -> None:
        processor = self.create_processor()
        processor.process(amount)

    @abstractmethod
    def create_processor(self) -> PaymentProcessor:
        ...

# ---------- Concrete Creators ----------
class USCheckoutService(CheckoutService):
    """В США пока запрещены крипто‑платежи."""
    def create_processor(self) -> PaymentProcessor:
        if self._provider_name == "crypto":
            raise ValueError("Crypto not supported in US yet")
        return _PROCESSOR_REGISTRY[self._provider_name]()

class EUEasyCheckoutService(CheckoutService):
    """В ЕС доступны все зарегистрированные процессоры."""
    def create_processor(self) -> PaymentProcessor:
        return _PROCESSOR_REGISTRY[self._provider_name]()

# ---------- Клиентский код ----------
if __name__ == "__main__":
    cart_total = 99.90

    # Покупатель из США
    us_checkout = USCheckoutService("stripe")
    us_checkout.pay(cart_total)     # → 💳 Stripe обработал $99.90

    # Покупатель из ЕС
    eu_checkout = EUEasyCheckoutService("crypto")
    eu_checkout.pay(cart_total)     # → ₿ Криптокошелёк получил эквивалент $99.90

    # Добавляем новый способ «ApplePay» без изменений в CheckoutService
    @register_processor("applepay")
    class ApplePayProcessor(PaymentProcessor):
        def process(self, amount: float) -> None:
            print(f"🍏 ApplePay списал ${amount:.2f}")

    eu_checkout2 = EUEasyCheckoutService("applepay")
    eu_checkout2.pay(10.0)


💳 Stripe обработал $99.90
₿ Криптокошелёк получил эквивалент $99.90
🍏 ApplePay списал $10.00
